In [1]:
import geopandas
import pandas as pd

In [2]:
dfo=pd.read_csv('properatti.csv')

In [10]:
dfo.columns

Index(['Unnamed: 0', 'operation', 'property_type', 'place_name',
       'place_with_parent_names', 'country_name', 'state_name', 'geonames_id',
       'lat-lon', 'lat', 'lon', 'price', 'currency',
       'price_aprox_local_currency', 'price_aprox_usd', 'surface_total_in_m2',
       'surface_covered_in_m2', 'price_usd_per_m2', 'price_per_m2', 'floor',
       'rooms', 'expenses', 'properati_url', 'description', 'title',
       'image_thumbnail'],
      dtype='object')

In [12]:
dfo.groupby('state_name').count()

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,geonames_id,lat-lon,lat,lon,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
state_name,,,,,,,,,,,,,,,,,,,,,
Bs.As. G.B.A. Zona Norte,25560,25560,25560,25537,25560,25560,15409,11438,11438,11438,...,23788,17405,22518,1787,6940,3081,25560,25559,25560,25120
Bs.As. G.B.A. Zona Oeste,9322,9322,9322,9322,9322,9322,7941,5617,5617,5617,...,7940,4623,6266,570,4790,633,9322,9322,9322,9032
Bs.As. G.B.A. Zona Sur,13952,13952,13952,13952,13952,13952,9516,5969,5969,5969,...,10940,6173,9907,278,2884,1777,13952,13952,13952,13499
Buenos Aires Costa Atlántica,10006,10006,10006,10006,10006,10006,8937,8205,8205,8205,...,7535,6350,6924,163,7646,440,10006,10006,10006,9907
Buenos Aires Interior,2291,2291,2291,2291,2291,2291,2154,1547,1547,1547,...,1780,1011,1441,26,940,85,2291,2291,2291,2270
Capital Federal,32316,32316,32316,32316,32316,32316,31030,23770,23770,23770,...,29977,23434,27324,3009,15325,6592,32316,32316,32316,31828
Catamarca,27,27,27,27,27,27,24,3,3,3,...,21,6,21,1,3,0,27,27,27,26
Chaco,57,57,57,57,57,57,57,40,40,40,...,33,28,28,0,17,1,57,57,57,57
Chubut,259,259,259,259,259,259,230,53,53,53,...,107,125,50,1,24,1,259,259,259,208


In [13]:
dfgeo=dfo.loc[dfo['state_name']=='Capital Federal',['place_name','lat','lon']]
dfgeo.head()

,place_name,lat,lon
0,Mataderos,-34.661824,-58.508839
2,Mataderos,-34.652262,-58.522982
3,Liniers,-34.647797,-58.516424
7,Belgrano,-34.559873,-58.443362
8,Belgrano,-34.559873,-58.443362


In [16]:
barrios=pd.read_csv('barrios.csv', encoding='latin1')


In [30]:
import shapely.wkt
barrios["coordinates"]= barrios['WKT'].apply(shapely.wkt.loads)

wktext=barrios.loc[0,'WKT']
shapetext=barrios.loc[0,'coordinates']
print(type(wktext))
print(type(shapetext))


<class 'str'>
<class 'shapely.geometry.polygon.Polygon'>


In [36]:
gbarrios = geopandas.GeoDataFrame(barrios, geometry='coordinates')


In [40]:
gbarrios.head()

,WKT,BARRIO,COMUNA,PERIMETRO,AREA,coordinates
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,..."
1,"POLYGON ((-58.4655768128541 -34.5965577078058,...",PATERNAL,15.0,7087.513295,2.229829e+06,"POLYGON ((-58.4655768128541 -34.5965577078058,..."
2,"POLYGON ((-58.4237529813037 -34.5978273383243,...",VILLA CRESPO,15.0,8132.699348,3.613584e+06,"POLYGON ((-58.4237529813037 -34.5978273383243,..."
3,"POLYGON ((-58.4946097568899 -34.6148652395239,...",VILLA DEL PARQUE,11.0,7705.389797,3.399596e+06,"POLYGON ((-58.4946097568899 -34.6148652395239,..."
4,"POLYGON ((-58.4128700313089 -34.6141162515854,...",ALMAGRO,5.0,8537.901368,4.050752e+06,"POLYGON ((-58.4128700313089 -34.6141162515854,..."


In [42]:
from shapely.geometry import Point
dfgeo = geopandas.GeoDataFrame(dfgeo, geometry=[Point(xy) for xy in zip(dfgeo['lon'], dfgeo['lat'])])

In [44]:
gkba_string="+proj=tmerc +lat_0=-34.629269 +lon_0=-58.4633 +k=0.9999980000000001 +x_0=100000 +y_0=100000 +ellps=intl +units=m +no_defs"
dfgeo.crs=gkba_string
gbarrios.crs=gkba_string

In [45]:
df=geopandas.sjoin(gbarrios,dfgeo, how='left', op='within')
df

RTreeError: Coordinates must not have minimums more than maximums

In [49]:
dfgeo.dropna(inplace=True)

In [57]:
df=geopandas.sjoin(gbarrios,dfgeo, how='left')


In [61]:
df.columns

Index(['WKT', 'BARRIO', 'COMUNA', 'PERIMETRO', 'AREA', 'coordinates',
       'index_right', 'place_name', 'lat', 'lon'],
      dtype='object')

In [68]:
df['place_name']=df['place_name'].str.upper()

In [72]:
df[df['BARRIO']==df['place_name']]

,WKT,BARRIO,COMUNA,PERIMETRO,AREA,coordinates,index_right,place_name,lat,lon
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,...",11033,CHACARITA,-34.583265,-58.452488
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,...",62583,CHACARITA,-34.583096,-58.453802
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,...",35770,CHACARITA,-34.583079,-58.448097
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,...",88799,CHACARITA,-34.582940,-58.448682
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,...",18998,CHACARITA,-34.582464,-58.452208
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,...",100282,CHACARITA,-34.581661,-58.451573
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,...",95220,CHACARITA,-34.581635,-58.454717
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,...",117955,CHACARITA,-34.580907,-58.454178
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,...",108906,CHACARITA,-34.580776,-58.454171
0,"POLYGON ((-58.4528200492791 -34.5959886570639,...",CHACARITA,15.0,7725.695228,3.118101e+06,"POLYGON ((-58.4528200492791 -34.5959886570639,...",42566,CHACARITA,-34.580718,-58.450785
